<a href="https://colab.research.google.com/github/harshitharavi26/DIP/blob/master/Sketch_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Model
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from sklearn.datasets import load_files
import numpy as np
from sklearn.model_selection import  train_test_split
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import cv2
from keras import applications
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

In [ ]:
INIT_LR = 1e-3
width=256
height=256
depth=3

In [ ]:
# upload the dataset to your drive and give its path
import glob
img=glob.glob(("/content/drive/My Drive/PNG/*"))
for i in img:
    print(i)
# this will read the dataset and display all the folder names

/content/drive/My Drive/PNG/wineglass
/content/drive/My Drive/PNG/wine-bottle
/content/drive/My Drive/PNG/wheel
/content/drive/My Drive/PNG/wrist-watch
/content/drive/My Drive/PNG/windmill
/content/drive/My Drive/PNG/walkie talkie
/content/drive/My Drive/PNG/wheelbarrow
/content/drive/My Drive/PNG/zebra
/content/drive/My Drive/PNG/vase
/content/drive/My Drive/PNG/violin
/content/drive/My Drive/PNG/truck
/content/drive/My Drive/PNG/trumpet
/content/drive/My Drive/PNG/train
/content/drive/My Drive/PNG/trousers
/content/drive/My Drive/PNG/tree
/content/drive/My Drive/PNG/traffic light
/content/drive/My Drive/PNG/tv
/content/drive/My Drive/PNG/umbrella
/content/drive/My Drive/PNG/van
/content/drive/My Drive/PNG/trombone
/content/drive/My Drive/PNG/tiger
/content/drive/My Drive/PNG/toothbrush
/content/drive/My Drive/PNG/tent
/content/drive/My Drive/PNG/toilet
/content/drive/My Drive/PNG/tomato
/content/drive/My Drive/PNG/tire
/content/drive/My Drive/PNG/tennis-racket
/content/drive/My Drive

In [ ]:
# dir will have the path
dir = '/content/drive/My Drive/PNG'

def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    target_labels = np.array(data['target_names'])
    return files,targets,target_labels
    
x, y,target_labels = load_dataset(dir)
print('Loading complete!')
print('X contains: ')
print(x)
print('Y contains: ')
print(y)
print('target_labels Contains: ')
print(target_labels)

Loading complete!
X contains: 
['/content/drive/My Drive/PNG/octopus/11217.png'
 '/content/drive/My Drive/PNG/door/5524.png'
 '/content/drive/My Drive/PNG/sponge bob/16008.png' ...
 '/content/drive/My Drive/PNG/lion/9844.png'
 '/content/drive/My Drive/PNG/panda/11478.png'
 '/content/drive/My Drive/PNG/bush/2733.png']
Y contains: 
[140  69 200 ... 123 143  34]
target_labels Contains: 
['airplane' 'alarm clock' 'angel' 'ant' 'apple' 'arm' 'armchair' 'ashtray'
 'axe' 'backpack' 'banana' 'barn' 'baseball bat' 'basket' 'bathtub'
 'bear (animal)' 'bed' 'bee' 'beer-mug' 'bell' 'bench' 'bicycle'
 'binoculars' 'blimp' 'book' 'bookshelf' 'boomerang' 'bottle opener'
 'bowl' 'brain' 'bread' 'bridge' 'bulldozer' 'bus' 'bush' 'butterfly'
 'cabinet' 'cactus' 'cake' 'calculator' 'camel' 'camera' 'candle' 'cannon'
 'canoe' 'car (sedan)' 'carrot' 'castle' 'cat' 'cell phone' 'chair'
 'chandelier' 'church' 'cigarette' 'cloud' 'comb' 'computer monitor'
 'computer-mouse' 'couch' 'cow' 'crab' 'crane (machine

In [ ]:
no_of_classes = len(np.unique(y))
print(no_of_classes)
# this is used in that last layers of the model

243


In [ ]:
#dividing the entire dataset into training data and validation data
X_train,x_val,Y_train,y_val = train_test_split(x,y,test_size=0.18,random_state=42)
#dividing the training dataset into training and testing
X_train,x_test,Y_train,y_test = train_test_split(X_train,Y_train,test_size=0.22,random_state=42)
X_train.shape,Y_train.shape,x_val.shape,y_val.shape,x_test.shape,y_test.shape
# sum should result in total
print(len(X_train))
print(len(x_val))
print(len(x_test))
print(X_train[0])
print(Y_train[0])
print(x_test[0])
print(y_test[0])

12358
3478
3486
/content/drive/My Drive/PNG/book/1991.png
24
/content/drive/My Drive/PNG/key/9265.png
115


In [ ]:
default_image_size=tuple((width,height))
def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
         # Convert to Numpy Array
        image=cv2.imread(file)
        if image is not None :
          image=cv2.resize(image, default_image_size) 
          images_as_array.append(img_to_array(image))
    return images_as_array

X_train = np.array(convert_image_to_array(X_train))
print('Training set shape : ',X_train.shape)

x_val = np.array(convert_image_to_array(x_val))
print('Validation set shape : ',x_val.shape)

x_test = np.array(convert_image_to_array(x_test))
print('Test set shape : ',x_test.shape)

print('1st training image shape ',X_train[0].shape)

NameError: ignored

In [ ]:
print('1st training image as array',X_train[0]) 

In [ ]:
X_train = X_train.astype('float32')/255
x_val = x_val.astype('float32')/255
x_test = x_test.astype('float32')/255
X_train[0]

In [ ]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (width, height, depth))
#loading the VGG19 model

80142336/80134624 [==============================] - 0s 0us/step


In [ ]:
for layer in model.layers[:5]:
    layer.trainable = False

In [ ]:
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(no_of_classes, activation="softmax")(x)

# creating the final model by updating last few layers of the VGG19 model
model_final = Model(model.input, predictions)
model_final.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)      

In [ ]:
aug = ImageDataGenerator(
    rotation_range=30, width_shift_range=0.15,
    height_shift_range=0.15, shear_range=0.15, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")
aug1=ImageDataGenerator(
    rotation_range=30, width_shift_range=0.15,
    height_shift_range=0.15, shear_range=0.15,
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
print('Compiled!')

In [ ]:
#batch_size = 32
checkpoint = ModelCheckpoint(filepath = 'cnn1.hdf5',monitor='val_accuracy' ,verbose = 1, save_best_only = True)
reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=2,min_lr=0.00001,mode='auto')
#checkpointer = ModelCheckpoint('cnn.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='auto')
history = model.fit_generator(
        aug.flow(X_train,Y_train,batch_size=32),
        #steps_per_epoch=80,
        epochs=50,
        validation_data=aug1.flow(x_val, y_val,batch_size=32),
        #validation_steps=50,
        callbacks=[checkpoint,reduce_lr],
        verbose=2, 
        shuffle=True
        )

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.legend(['acc', 'val_acc'], loc='lower right')  
plt.title('Training and validation accuracy')


plt.figure()
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.legend(['loss', 'val_loss'], loc='upper right')  
plt.title('Training and validation loss')

plt.show()

In [ ]:
model.load_weights('cnn1.hdf5')

In [ ]:
# evaluate and print test accuracy
score = model.evaluate(x_test, y_test,batch_size=32,verbose=0)
print('\n', 'Test Loss','Test accuracy:', score[0],score[1])
#accuracy !!